In [28]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn import svm
import matplotlib.pyplot as plt

def eval_tree_based_model_max_depth(clf, max_depth, X_train, y_train, X_test, y_test):
    """
    This function evaluates the given classifier (either a decision tree or random forest) at all of the 
    maximum tree depth parameters in the vector max_depth, using the given training and testing
    data. It returns two vector, with the training and testing classification errors.
    
    Inputs:
        clf: either a decision tree or random forest classifier object
        max_depth: a (T, ) vector of all the max_depth stopping condition parameters 
                            to test, where T is the number of parameters to test
        X_train: (N, D) matrix of training samples.
        y_train: (N, ) vector of training labels.
        X_test: (N, D) matrix of test samples
        y_test: (N, ) vector of test labels
    Output:
        train_err: (T, ) vector of classification errors on the training data
        test_err: (T, ) vector of classification errors on the test data
    """
    training_errors = []
    test_errors = []
    # evaluates the tree classifier for the desired number of max_tree_depth values values 
    for i in range(len(max_depth)):
        clf.set_params(min_samples_leaf = max_depth[i])
        clf.fit(X_train, y_train)
        y_train_predict = clf.predict_proba(X_train)[:, 1]
        y_test_predict = clf.predict_proba(X_test)[:,1]
        
        training_errors.append(roc_auc_score(y_train, y_train_predict))
        test_errors.append(roc_auc_score( y_test, y_test_predict))
    return np.array(training_errors), np.array(test_errors)

def classification_err(y, real_y):
    """
    This function returns the classification error between two equally-sized vectors of 
    labels; this is the fraction of samples for which the labels differ.
    
    Inputs:
        y: (N, ) shaped array of predicted labels
        real_y: (N, ) shaped array of true labels
    Output:
        Scalar classification error
    """
    return sum(y != real_y)/len(y)



In [ ]:
data = np.loadtxt(open("train_2008.csv", "rb"), delimiter=",", skiprows=1)
test_data = np.loadtxt(open("test_2008.csv", "rb"), delimiter=",", skiprows=1)

kf = KFold(n_splits = 5) 

X = data[:, 3:382]
y = data[:, 382]

X_test = data[:, 3:382]



all_predictions = np.zeros(5)
for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_val = X[train_index], X[test_index]
    y_train, y_val = y[train_index], y[test_index]
    
#     #fprint(np.shape(y_test))
#     scaler = StandardScaler()
#     # Fit on training set only.
#     scaler.fit(X_train)
#     # Apply transform to both the training set and the test set.
#     X_train = scaler.transform(X_train)
#     X_val = scaler.transform(X_val)
    
    '''pca = PCA(n_components = 10)
    pca.fit(X_train)
    X_train = pca.transform(X_train)
    X_val = pca.transform(X_val)'''
    
    clf = RandomForestClassifier(min_samples_leaf = 101, n_estimators=100)
#   clf = svm.SVC(probability = True)
    clf.fit(X_train, y_train)
    print(clf.classes_)
    
    
    predictions = clf.predict_proba(X_val)[:, 1]
    print(sum(np.floor(2.0*predictions) != np.array(y_val))/len(predictions))
    print(roc_auc_score(y_val, predictions))
    
    max_depth = np.arange(1, 10000, 100)
    train_err, test_err = eval_tree_based_model_max_depth(clf, max_depth, X_train, 
                                                            y_train, X_val, y_val)

    plt.figure()
    plt.plot(max_depth, test_err, label='Testing error')
    plt.plot(max_depth, train_err, label='Training error')
    plt.xlabel('Maximum Tree Depth')
    plt.ylabel('Classification error')
    plt.title('Decision Tree with Gini Impurity and Maximum Tree Depth')
    plt.legend(loc=0, shadow=True, fontsize='x-large')
    plt.show()

    print('Test error minimized at max_depth = %i' % max_depth[np.argmax(test_err)])

